<a href="https://colab.research.google.com/github/sren97/Proyecto-Modelos/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os

# **Interacción con los Archivos**

## **Descargar archivos CSV**

Con este código se descargan los archivos directamente desde la plataforma Kaggle mediante la API

In [ ]:
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 25% 5.00M/20.1M [00:00<00:00, 50.2MB/s]
100% 20.1M/20.1M [00:00<00:00, 104MB/s] 


## **Descomprimir los archivos**

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


#**Cargar archivos como Dataframes**

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
auxiliar_df = train_df.copy()

In [ ]:
train_df.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'ESTU_PAGOMATRICULAPROPIO',
       'RENDIMIENTO_GLOBAL'],
      dtype='object')

# **Limpieza de Datos**


## **Funciones de Limpieza**

### **Rendimiento Global**

In [ ]:
def rendimientoglobal(df):
  map_dict = {
    "alto":3,
    "medio-alto":2,
    "medio-bajo":1,
    "bajo": 0
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)

### **Rendimiento Global Reverso**

In [ ]:
def rendimientoglobalreverse(df):
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].round().astype(int)
  map_dict = {
    3:"alto",
    2:"medio-alto",
    1:"medio-bajo",
    0:"bajo"
  }
  df['RENDIMIENTO_GLOBAL'] = df['RENDIMIENTO_GLOBAL'].map(map_dict)

### **Estrato**

In [ ]:
def estrato(df):

  moda = df['FAMI_ESTRATOVIVIENDA'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].apply(generar_valores)


### **Internet**

In [ ]:
#NUEVO - la probabilidad no es una constante, es el promedio de personas que dijieron que si
import random
def internet(df):

  percent = np.round(df['FAMI_TIENEINTERNET'].value_counts(normalize=True).get('Si', 0), 4)

  def assign_value(row):
    est_value=row['FAMI_ESTRATOVIVIENDA']
    if pd.isnull(row['FAMI_TIENEINTERNET']):
        if ((est_value != 'Sin Estrato') or (est_value != 'Estrato 1') or (est_value != 'Estrato 2')) :
            return 'Si'
        elif np.round(random.random(),4) <= percent:
            return 'Si'
        else:
            return 'No'
    else:
        return row['FAMI_TIENEINTERNET']

  df['FAMI_TIENEINTERNET'] = df.apply(assign_value, axis=1)

### **Matricula Propia**

In [ ]:
def matriculapropia(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PAGOMATRICULAPROPIO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores_normales(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PAGOMATRICULAPROPIO'] = df['ESTU_PAGOMATRICULAPROPIO'].apply(generar_valores_normales)


### **Horas que Trabaja**

In [ ]:
def horassemanatrabaja(df):

  moda = df['ESTU_HORASSEMANATRABAJA'].mode()[0]

  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].apply(generar_valores)

### **Valor de la Matricula**

In [ ]:
def valormatriculauniversidad(df):

  moda = df['ESTU_VALORMATRICULAUNIVERSIDAD'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].apply(generar_valores)

### **Educación de Padre**

In [ ]:
def educacionpadre(df):

  moda = df['FAMI_EDUCACIONPADRE'].mode()[0]

  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].apply(generar_valores)


### **Educación de Madre**

In [ ]:
def educacionmadre(df):

  moda = df['FAMI_EDUCACIONMADRE'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].apply(generar_valores)

### **Periodo**

In [ ]:
def periodo(df):

  moda = df['PERIODO'].mode()[0]
  # Función para generar valores aleatorios con la misma media y desviación estándar
  def generar_valores(row):
      if pd.isna(row):  # Si el valor es nulo
          return moda
      return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
  df['PERIODO'] = df['PERIODO'].apply(generar_valores)

### **Departamento**

In [ ]:
def departamento(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PRGM_DEPARTAMENTO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_DEPARTAMENTO'] = df['ESTU_PRGM_DEPARTAMENTO'].apply(generar_valores)


### **Añadir Nuevas Columnas de Relación**

In [ ]:
def nuevascolumnas(df):
    df['RELACION_PROGRAMA_VALOR_MATRICULA'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_VALORMATRICULAUNIVERSIDAD']}", axis=1)

    df['RELACION_PROGRAMA_VALOR_MATRICULA_DEPARTAMENTO'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_VALORMATRICULAUNIVERSIDAD']}_{x['ESTU_PRGM_DEPARTAMENTO']}", axis=1)

    df['RELACION_PROGRAMA_DEPARTAMENTO'] = df.apply(lambda x: f"{x['ESTU_PRGM_ACADEMICO']}_{x['ESTU_PRGM_DEPARTAMENTO']}", axis=1)

    df.drop(columns=['ESTU_PRGM_ACADEMICO'], inplace=True)

### **Programa Acádemico**

In [ ]:
def programaacademico(df):

    # Agrupar por departamento y calcular la media y desviación estándar del rendimiento global
    moda = df['ESTU_PRGM_ACADEMICO'].mode()[0]

    # Función para generar valores aleatorios con la misma media y desviación estándar
    def generar_valores(row):
        if pd.isna(row):  # Si el valor es nulo
            return moda
        return row

    # Aplicar la función para sustituir los valores nulos con una distribución normal equivalente
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(generar_valores)


## **Limpieza Total**

In [ ]:
def limpiar_df(df, predict=False, test=False):
  if not predict:
    if not test:
      rendimientoglobal(df)
    estrato(df)
    internet(df)
    matriculapropia(df)
    valormatriculauniversidad(df)
    educacionpadre(df)
    educacionmadre(df)
    horassemanatrabaja(df)
    departamento(df)
    programaacademico(df)
    periodo(df)
    nuevascolumnas(df)
    return df
  elif predict:
    rendimientoglobal(df)

In [ ]:
limpiar_df(auxiliar_df, predict=True)
train_df2 = limpiar_df(train_df)
test_df2 = limpiar_df(test_df, test=True)

# **Catboost Classifier**

In [ ]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.4 MB/s eta 0:00:00


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar los datos
data = train_df2.copy()  # train_df_clean

# Dividir datos en características y variable objetivo
X = data.drop(columns=["ID", "RENDIMIENTO_GLOBAL"])
y = data["RENDIMIENTO_GLOBAL"]

categorical_features = X.select_dtypes(include=['object']).columns.tolist()

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=16, stratify=y)

# Crear el modelo CatBoost con bootstrap_type='Bayesian'
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.07,
    depth=6,
    l2_leaf_reg=3,
    bootstrap_type='Bernoulli',
    subsample=0.75,
    auto_class_weights='Balanced',
    thread_count=4,
    verbose=100,
    task_type='CPU',
    eval_metric='Accuracy',
)

# Entrenar el modelo
model.fit(X_train, y_train, eval_set=(X_test, y_test), cat_features=categorical_features, use_best_model=True)

# Predecir en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy}")

0:	learn: 0.4007380	test: 0.4044415	best: 0.4044415 (0)	total: 5.79s	remaining: 1h 36m 28s
100:	learn: 0.4380887	test: 0.4396216	best: 0.4396216 (100)	total: 9m 20s	remaining: 1h 23m 6s
200:	learn: 0.4420092	test: 0.4435932	best: 0.4438235 (191)	total: 19m 11s	remaining: 1h 16m 18s
300:	learn: 0.4447257	test: 0.4457849	best: 0.4457849 (300)	total: 29m 10s	remaining: 1h 7m 46s
400:	learn: 0.4461590	test: 0.4458574	best: 0.4460323 (334)	total: 39m 8s	remaining: 58m 27s
500:	learn: 0.4473133	test: 0.4465646	best: 0.4466812 (498)	total: 49m 7s	remaining: 48m 55s
600:	learn: 0.4485849	test: 0.4465558	best: 0.4469274 (550)	total: 58m 55s	remaining: 39m 6s
700:	learn: 0.4495997	test: 0.4468739	best: 0.4469443 (675)	total: 1h 8m 39s	remaining: 29m 17s
800:	learn: 0.4505587	test: 0.4468306	best: 0.4469868 (768)	total: 1h 18m 22s	remaining: 19m 28s
900:	learn: 0.4515377	test: 0.4463372	best: 0.4470910 (812)	total: 1h 28m 7s	remaining: 9m 40s
999:	learn: 0.4525128	test: 0.4466690	best: 0.4470910 

# **Predicción Kaggle**

In [ ]:
pred=test_df2.copy()
y = pred["ID"]
pred.drop(columns=["ID","Unnamed: 0"],axis=1,inplace=True)

predicciones = model.predict(pred)
sol=pd.concat([y,pd.DataFrame(predicciones,columns=["RENDIMIENTO_GLOBAL"])],axis=1)

rendimientoglobalreverse(sol)

sol.to_csv("submission.csv",index=False)